<a href="https://colab.research.google.com/github/IndujaSuresh/Deep-learning/blob/main/deep_cv_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.datasets import load_iris

# Define MLP class
class MLP:
    def __init__(self, input_size, hidden_sizes, output_size):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.W1 = np.random.randn(input_size, hidden_sizes[0])
        self.b1 = np.zeros((1, hidden_sizes[0]))
        self.W2 = np.random.randn(hidden_sizes[0], output_size)
        self.b2 = np.zeros((1, output_size))
        if len(hidden_sizes) > 1:
            self.hidden_weights = []
            self.hidden_biases = []
            for i in range(1, len(hidden_sizes)):
                self.hidden_weights.append(np.random.randn(hidden_sizes[i-1], hidden_sizes[i]))
                self.hidden_biases.append(np.zeros((1, hidden_sizes[i])))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self, X):
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self.sigmoid(self.z1)
        if len(self.hidden_sizes) > 1:
            for i in range(len(self.hidden_sizes) - 1):
                z = np.dot(self.a1, self.hidden_weights[i]) + self.hidden_biases[i]
                a = self.sigmoid(z)
                self.a1 = a
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        y_hat = self.sigmoid(self.z2)
        return y_hat

    def compute_loss(self, X, y):
        y_hat = self.forward(X)
        loss = np.mean((y - y_hat)**2)
        return loss

    def predict(self, X):
        y_hat = self.forward(X)
        return np.round(y_hat)

    def get_params(self):
        params = {
            "W1": self.W1,
            "b1": self.b1,
            "W2": self.W2,
            "b2": self.b2
        }
        if len(self.hidden_sizes) > 1:
            for i in range(len(self.hidden_sizes) - 1):
                params[f"W{i+2}"] = self.hidden_weights[i]
                params[f"b{i+2}"] = self.hidden_biases[i]
        return params

    def set_params(self, params):
        self.W1 = params["W1"]
        self.b1 = params["b1"]
        self.W2 = params["W2"]
        self.b2 = params["b2"]
        if len(self.hidden_sizes) > 1:
            for i in range(len(self.hidden_sizes) - 1):
                self.hidden_weights[i] = params[f"W{i+2}"]
                self.hidden_biases[i] = params[f"b{i+2}"]


# Define Tournament Selection GA class
class TSGA:
    def __init__(self, population_size, mutation_rate, num_genes):
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.num_genes = num_genes
        self.population = np.random.uniform(low=-1, high=1, size=(population_size, num_genes))
        self.fitness_scores = None

    def _fitness(self, X, y, weights):
        mlp = MLP(4, [8], 1)
        mlp.set_params({
            "W1": weights[:32].reshape(4, 8),
            "b1": weights[32:40].reshape(1, 8),
            "W2": weights[40:48].reshape(8, 1),
            "b2": weights[48:]
        })
        loss = mlp.compute_loss(X, y)
        return 1 / (loss + 1)

    def _tournament_selection(self, tournament_size):
        tournament_indices = np.random.choice(range(self.population_size), size=tournament_size, replace=False)
        tournament = self.population[tournament_indices]
        fitness_scores = np.array([self._fitness(X, y, weights) for weights in tournament])
        winner_index = np.argmax(fitness_scores)
        winner = tournament[winner_index]
        return winner

    def evolve(self, X, y, num_generations, tournament_size):
        for gen in range(num_generations):
            new_population = []
            for i in range(self.population_size):
                parent1 = self._tournament_selection(tournament_size)
                parent2 = self._tournament_selection(tournament_size)
                child = np.empty_like(parent1)
                for j in range(self.num_genes):
                    rand = np.random.rand()
                    if rand < self.mutation_rate:
                        child[j] = np.random.uniform(low=-1, high=1)
                    else:
                        if np.random.rand() < 0.5:
                            child[j] = parent1[j]
                        else:
                            child[j] = parent2[j]
                new_population.append(child)
            self.population = np.array(new_population)
            self.fitness_scores = np.array([self._fitness(X, y, weights) for weights in self.population])

    def get_best_weights(self, X, y):
        best_index = np.argmax(self.fitness_scores)
        best_weights = self.population[best_index]
        return {
            "W1": best_weights[:32].reshape(4, 8),
            "b1": best_weights[32:40].reshape(1, 8),
            "W2": best_weights[40:48].reshape(8, 1),
            "b2": best_weights[48:]
        }



In [ ]:
# Load Iris dataset
data = load_iris()
X, y = data.data, data.target
y = y.reshape(-1, 1)

# Create TSGA object and evolve population
tsga = TSGA(population_size=100, mutation_rate=0.05, num_genes=49)
tsga.evolve(X, y, num_generations=100, tournament_size=20)

# Get best MLP weights and use them to make predictions
best_weights = tsga.get_best_weights(X, y)
mlp = MLP(4, [8], 1)
mlp.set_params(best_weights)
y_pred = mlp.predict(X)
accuracy = np.mean(y_pred == y)
print(f"Accuracy: {accuracy}")


Accuracy: 0.6666666666666666
